In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, lower, regexp_replace
import os

os.environ['SPARK_HOME']="/opt/homebrew/Cellar/apache-spark/3.5.3/libexec"
os.environ['PYSPARK_PYTHON']='python'

# Создание SparkSession
spark = SparkSession.builder.appName("Movie Word Count with Preprocessing").getOrCreate()

# Чтение данных из CSV-файла
df = spark.read.csv("./movies.csv", header=True)

# Предобработка и извлечение слов
words_df = df.select(
    explode(
        split(
            regexp_replace(lower(col("Film")), r"[^a-zA-Z0-9\s]", ""),  # Удаление нежелательных символов
            r"\s+"  # Разделение по пробелам
        )
    ).alias("word")
)

# Исключение стоп-слов
stop_words = [
    "the", "and", "a", "of", "to", "in", "is", "it", "you", "that",
    "for", "on", "with", "as", "this", "by"
]
filtered_words_df = words_df.filter(~col("word").isin(stop_words))

# Подсчет частоты слов
word_counts = filtered_words_df.groupBy("word").count()

# Нахождение самого частого слова
most_common_word = word_counts.orderBy(col("count").desc()).first()

if most_common_word:
    print(f"Самое частое слово: '{most_common_word['word']}', которое встречается {most_common_word['count']} раз).")
else:
    print("Не найдено ни одного слова после фильтрации.")

# Остановка SparkSession
spark.stop()

Самое частое слово 'love' которое встречается 4 раз.
